## Разработка модели машиного обучения для опознания вредоносных сайтов

### Загрузка данных

Загрузим необходимые библиотеки.

In [1]:
!pip install razdel -q

Импортируем необходимые библиотеки

In [2]:
import pandas as pd
import numpy as np

import razdel

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

Загрузим датасеты.

In [ ]:
df_train = pd.read_csv('data/train.csv') 
df_test = pd.read_csv('data/test.csv') 

Проверим есть ли пропуски в тренировочном датасете

In [5]:
print(f'Количество пропусков в тренировочных данных \n{df_train.isna().sum()}\n')
print(f'Количество пропусков в тестовых данных \n{df_test.isna().sum()}')

Количество пропусков в тренировочных данных 
ID       0
url      0
title    1
label    0
dtype: int64

Количество пропусков в тестовых данных 
ID       0
url      0
title    0
dtype: int64


В данных присутсвуют пропуски, обработаем их в пайплайне

Разделим данные на тренировочную и валидационную выборки



Выделим зависимые и независимые переменные.

In [6]:
X = df_train[['url','title']]
y = df_train[['label']]

Разделим данные на тренировочную и валидационную выборки


In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42,stratify=y)

### Модель: Обработка текста - токенизация и стемминг, логистическая регрессия

Опишем класс трансформера в пайплайне, который принимает датасет с двумя строковыми столбцами и возвращеает одну объединённую строку.

In [8]:
class CombineColumns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        title_column = X[:, 0]  # Первый столбец (title)
        url_column = X[:, 1]    # Второй столбец (url)

        # Объединяем столбцы 'title' и 'url' в одну строку
        return (title_column + ' ' + url_column)

Опишем класс трансформера в пайплайне, который будет принимать датасет состоящий из столбца со строками, удалять все стоп слова английского и русского языка, и производить стемматизацию

In [9]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words_r = set(stopwords.words("russian"))
stop_words_e = set(stopwords.words("english"))

class SnowballStemmerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stemmer = SnowballStemmer("russian")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        stemmed_texts = []
        for text in X:
            # Токенизируем текст с помощью ntkl
            tokens = nltk.tokenize.casual_tokenize(text)

            # Уберем русские и английские стоп слова
            filtered_tokens = [token for token in tokens if token.lower() not in stop_words_r and token.lower() not in stop_words_e]

            # Выполним стемминг с помощью SnowballStemmer
            stemmed_words = [self.stemmer.stem(token) for token in filtered_tokens]

            # Собираем слова обратно в строку
            stemmed_text = ' '.join(stemmed_words)
            stemmed_texts.append(stemmed_text)

        return pd.Series(stemmed_texts)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Опишем класс трансформера в пайплайне, который будет заменять пустые значения в датасете на пустые строки.

In [10]:
preprocessor = ColumnTransformer(
    transformers=[
        ('title_url', SimpleImputer(strategy='constant', fill_value=''), ['title','url'])
    ],
    remainder='passthrough'
)

Опишем сам пайплайн.

In [11]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Применяем preprocessor для обработки пропусков
    ('combine', CombineColumns()),  # Объединяем столбцы 'title' и 'url' в одну строку
    ('stemmer', SnowballStemmerTransformer()),  # Стеммер для русского языка
    ('vectorizer', TfidfVectorizer(stop_words='english', max_features=3000)),  # Векторизация текста
    ('classifier', LogisticRegression())  # Классификатор
])

Создадим метрику для поиска в gridSearch

In [12]:
f1_scorer = make_scorer(f1_score, average='weighted')

Опишем словарь параметров для поиска по сетке

In [13]:
param_grid = {
    # Выбор векторизатора
    'vectorizer': [TfidfVectorizer(max_features=8000, stop_words='english')],

    # Параметры для LogisticRegression
    'classifier__C': [25],  # Регуляризация
    'classifier__penalty': ['l2'],
    'classifier__solver': ['liblinear']
}

Произведем поиск по сетке и выведем модель с лучшими параметрами

In [14]:
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring=f1_scorer, cv=3)
grid_search.fit(X_train, y_train.values.ravel())

# Лучшие параметры и лучший результат
print("Лучшие параметры:", grid_search.best_params_)
print("Лучший результат по F1:", grid_search.best_score_)

Лучшие параметры: {'classifier__C': 25, 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear', 'vectorizer': TfidfVectorizer(max_features=8000, stop_words='english')}
Лучший результат по F1: 0.9918486893378257


Сохраним лучшую модель

In [15]:
best_model = grid_search.best_estimator_

Оценим метрику качества модели на валидационной выборке

In [16]:
y_pred = best_model.predict(X_valid)
f1 = f1_score(y_valid, y_pred)
print(f1)

0.9650986342943855


### Предсказание на тестовой выборке и сохранение результатов

Сделаем предсказание с помощью лучшей модели

In [17]:
test_predictions =  best_model.predict(df_test[['title', 'url']])

Сохраним предсказание в датафрейм

In [18]:
submission = pd.DataFrame({
    'ID': df_test['ID'],
    'label': test_predictions
})

Преобразуем в файл формата csv

In [19]:
submission.to_csv('submission.csv', index=False)

### Вывод

В рамках данного проекта была разработана модель машинного обучения для классификации вредоносных и безопасных веб-сайтов на основе их URL-адресов и описания. Для преобразования текста в числовое представление использовались базовые методы обработки текстов (такие как TF-IDF и стемминг), а в качестве классификатора применялась логистическая регрессия.

Модель была обучена на размеченном датасете и продемонстрировала приемлемые результаты на валидационном наборе. Ключевая метрика качества F1-score показала, что логистическая регрессия способна эффективно различать вредоносные и безопасные ссылки.